In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def onMouse(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE:
        if flags & cv2.EVENT_FLAG_LBUTTON:
            cv2.circle(param[0], (x, y), 10, (255, 255, 255), -1)
            cv2.circle(param[1], (x, y), 10, (255, 255, 255), -1)
            cv2.imshow('dst', param[1])

In [3]:
src = cv2.imread('../segmentation_rsc/hand_paper.jpg')
mask = np.zeros(shape = src.shape[:2], dtype = np.uint8)
markers = np.zeros(shape = src.shape[:2], dtype = np.int32)
dst = src.copy()
cv2.imshow('dst', dst)

mode = cv2.RETR_EXTERNAL
method = cv2.CHAIN_APPROX_NONE

while True:
    cv2.setMouseCallback('dst', onMouse, [mask, dst])
    key = cv2.waitKey(30)
    
    if key == 0x18:# esc키 누르면 break
        break
    elif key == ord('r'):# r키 누르면 리셋 (ord() unicord로 변환해 줌)
        mask[:, :] = 0
        dst = src.copy()
        cv2.imshow('dst', dst)
    elif key == ord(' '):# 스페이스 바 누르면 시드를 설정하고 watershed 결과 출력
        # 시드 설정
        contours, hierarchy = cv2.findContours(mask, mode, method)
        print(f'len(contours)={len(contours)}')
        markers[:, :] = 0
        for i, cnt in enumerate(contours):
            cv2.drawContours(markers, [cnt], 0, i + 1, -1)
        # watershed
        cv2.watershed(src, markers)
        # 결과 그리기
        dst = src.copy()
        dst[markers == -1] = [0, 0, 255]# 경계선을 빨갛게 표시
        for i in range(len(contours)):
            r = np.random.randint(256)
            g = np.random.randint(256)
            b = np.random.randint(256)
            dst[markers == i + 1] = [b, g, r]
            dst = cv2.addWeighted(src, .5, dst, .5, 0)
            cv2.imshow('dst', dst)
cv2.destroyAllWindows()

len(contours)=2
len(contours)=3


error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:2563: error: (-27:Null pointer) NULL window: 'dst' in function 'cvSetMouseCallback'
